In [1]:
                    #Mongo DB Collection Descriptions#
##############################################################
##############################################################
'''
LabeledStatuses_Hurricane_C   ()
LabeledStatuses_Irma1_C   ()
LabeledStatuses_Irma2_C   ()
LabeledStatuses_MiscRelevant_C   ()
LabeledStatuses_MiscTechCompanies_C   ()
LabeledStatuses_Power_A   ()
LabeledStatuses_Sandy_K   ()
Statuses_Florence_A   ()
Statuses_Florence_C   ()
Statuses_Irma_A   ()
Statuses_Irma_C   ()
Statuses_Irma_K_26_media_subset   (subset of Doina's set of Irma tweets from the streaming API restrincted to tweets @ or 
                                                                            mentioning one of the 26 local media accounts)
Statuses_Irma_K_w_usertype   (Doina's set of Irma tweets from the streaming API including a field that has the predicted 
                                                                                      usertype from the Classification ML)
#do this one# Statuses_Irma_Scraped_Streamed   (Conbination of Statuses_Irma_K_26_media_subset and Tweets_Irma_Scraped with added fields 
                                                                to distinguish from which collection the tweet comes from)
Statuses_Maria_A   ()
Statuses_Maria_C   ()
Statuses_MiscClimateChange_A   ()
Statuses_MiscGlobal_A   ()
Statuses_MiscKnoxville_A   ()
Statuses_MiscPower_A   ()
Tweets_Irma_Scraped   (set of Irma tweets collected from Scrapy Tweet Crawler, based on having to do with one of the 26 
                                                                                                    local media accounts)

'''
print()

In [2]:
#change the settings block and run then go to second block

                    #Settings#
##############################################################
##############################################################

#name of the collection on mongodb under twitter
collection_name = 'Tweets_Irma_Scraped'   

#number of tweets to comprise the test set at most
num = 1000

#the name of the .xlsx you want to create
xlsx_name = 'test_Irma_Scraped_urlextract.xlsx'

#set to true if you want the test set to be comprised of only relevant tweets
remove_irrelevant = False

#set to true if you want the test set to only unique tweets
remove_duplicates = True

#set to true if you want the test set to not contain retweets
remove_retweets = True

#set to true if you want the test set to be comprised of only tweets in English
make_english_only = False

#set to true if you want the test set to be comprised of only tweets during the specified dates
make_only_in_time_frame = False
#date format '2019-01-01'
date_begin = '2017-09-01'
date_end = '2017-09-30'

#set to None if not needed
#otherwise set to list of keywords you want each tweet to contain one of ['governor', 'FEMA']
only_contain_keywords = None #['governor', 'FEMA', 'fema', 'Fema']

#set to None if not needed
#otherwise set to the name of the csv that the training set is comprised of and that you dont want overlapping tweets from
training_set = None #'opinion_new_train.csv'

#set to False if you want tweets from news media sources
#and set to True if you do not want them
remove_tweets_from_media = False

##############################################################
##############################################################
import sys
if '/home/nwest13/twitter/Zach/Tool_Box/Modules' not in sys.path:
    sys.path.insert(0, '/home/nwest13/twitter/Zach/Tool_Box/Modules')
from Text_Cleaner import tc
import Quick_Find
import csv
import pymongo
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from bson import ObjectId
from dateutil import parser
import random
import pickle
import gensim
from pd_doc2vec import doc2vec
import re
from datetime import datetime
import nltk
from nltk import *
from nltk.corpus import *

tc_class = tc()
model = gensim.models.doc2vec.Doc2Vec.load('/home/nwest13/twitter/Zach/Tool_Box/Relevant_Model/Relevance.model')

client = pymongo.MongoClient('da1.eecs.utk.edu')
if client['twitter'][collection_name].count_documents({}) < 1000000:
    coll = list(client['twitter'][collection_name].find())
else:
    coll = Quick_Find.get_df(client['twitter'][collection_name])

df_new_set = pd.DataFrame(columns=['Date', 'ID', 'Text'])
df_coll_set = pd.DataFrame(coll) 

### TESTING
df_coll_set = df_coll_set.head(5000)
###

if collection_name != "Statuses_Irma_Scraped_Streamed":
    remove_tweets_from_media = False
elif remove_tweets_from_media:
    df_coll_set = df_coll_set[df_coll_set['from_media'] == 0]
    
if training_set != None:
    df_training_set = pd.read_csv(training_set)
    print('Training Set')
    print(df_training_set.iloc[0])
    print()
    
print('Set from Collection')    
print(df_coll_set.iloc[0])
print()

Set from Collection
ID                                           913915921810456578
_id                                    5cdee0933339f6a5e5b23ba1
datetime                                    2017-09-29 23:58:30
has_media                                                   NaN
is_reply                                                   True
is_retweet                                                False
medias                                                      NaN
nbr_favorite                                                  1
nbr_reply                                                     0
nbr_retweet                                                   0
scraped                                                       1
streamed                                                      0
text                        So glad you were able to get there!
tweeting_to_username                                 10NewsWTSP
url                     /donna_baznik/status/913915921810456578
user_id             

In [3]:
#look at the print above and find the 4 fields that corespond to this data
#replace the strings below with the correct name in the data frame
##############################################################
##############################################################
#### For Collection Set ####
Date = 'datetime'  #the date of the tweet
Text = 'text'  #the tweets text
ID = 'ID'  #the tweets ID

#### For Training Set ####
Train_set_Text = 'Text' #the tweets text 
##############################################################
##############################################################

def conv_date_seconds( string ): 
    date = parser.parse(string)
    str_date = date.strftime("%Y-%m-%d")
    return int((datetime.strptime( str_date, "%Y-%m-%d" )).timestamp())

print(len(df_coll_set))
print()

print('Cleaning Text from Collection Set')
if remove_retweets == True and make_english_only == True:
    print('Removing retweets')
    print('Removing any tweets not in english')
    df_coll_set = tc_class.clean_df(df_coll_set, Text, ['english', 'retweet', 'dot', 'at_user', 'hashtag', 
                                                        'html_link', 'pic_link', 'lower', 'numbers', 'punc',
                                                        'stop_words', 'new_line']) 
    
if remove_retweets == False and make_english_only == True:
    print('Removing any tweets not in english')
    df_coll_set = tc_class.clean_df(df_coll_set, Text, ['english', 'dot', 'at_user', 'hashtag',  
                                                        'html_link', 'pic_link', 'lower', 'numbers', 'punc',
                                                        'stop_words', 'new_line'])

if remove_retweets == True and make_english_only == False:
    print('Removing retweets')
    df_coll_set = tc_class.clean_df(df_coll_set, Text, ['retweet', 'dot', 'at_user', 'hashtag',  
                                                        'html_link', 'pic_link', 'lower', 'numbers', 'punc',
                                                        'stop_words', 'new_line']) 
    
if remove_retweets == False and make_english_only == False:
    df_coll_set = tc_class.clean_df(df_coll_set, Text, ['dot', 'at_user', 'hashtag',  
                                                        'html_link', 'pic_link', 'lower', 'numbers', 'punc',
                                                        'stop_words', 'new_line'])

df_coll_set.reset_index(drop=True, inplace=True)
print()

#Removes tweets in collection set that dont contain keywords
if only_contain_keywords != None:
    remove_list = []
    print('Removing tweets that dont contain keywords')
    
    regex_keyword = re.compile(r"(" + r"|".join(only_contain_keywords) + ")")    
    df_temp = df_coll_set[Text].str.contains(regex_keyword)
    df_coll_set = df_coll_set[df_temp]
   
    print(len(df_coll_set))
    print()

#does not add irrelevant tweets to test set if set to True
if remove_irrelevant == True:
    print('Removing irrelevant tweets')
    
    df_relevant_train = pd.DataFrame(columns=['text', 'Is_Relevant_Or_Not'])

    df_relevant_train.at[0, 'text'] = "Not Relevant"
    df_relevant_train.at[0, 'Is_Relevant_Or_Not'] = "irrelevant"
    df_relevant_train.at[1, 'text'] = "Relevant"
    df_relevant_train.at[1, 'Is_Relevant_Or_Not'] = "relevant"

    print(df_relevant_train)

    # for i in df_coll_set.index:
    #     df_coll_set.at[i, 'Is_Relevant_Or_Not'] = ""

    x = doc2vec(df_relevant_train, 'text', ['Is_Relevant_Or_Not'], build=True, given=model)

    print('predicting relevance')
    df_coll_set['Is_Relevant_Or_Not'] = df_coll_set[Text].map(x.predict_text)    
    df_coll_set['Is_Relevant_Or_Not'] = df_coll_set['Is_Relevant_Or_Not'].apply(lambda x: x[0])  
    df_coll_set = df_coll_set[df_coll_set['Is_Relevant_Or_Not'] == 'relevant']
    
    print(len(df_coll_set))
    print()

    
#Removes retweets in collection set
if remove_retweets == True:
    print('Removing retweets')
    remove_list = []
    
    for i in tqdm(df_coll_set.index):
        if df_coll_set.at[i, Text].startswith('RT @'):
            remove_list.append(i)
        
    df_coll_set = df_coll_set.drop(remove_list)
    df_coll_set.reset_index(drop=True, inplace=True)   
    print(len(df_coll_set))
    print()

#Removes tweets in traing set from collection set if not set to None
if training_set != None:
    print('Removing tweets from collection set that are in Training Set')
    df_training_set[Train_set_Text] = df_training_set[Train_set_Text].astype(str)
    df_training_set = df_training_set[df_training_set[Train_set_Text] != 'nan']
    df_training_set.reset_index(inplace=True, drop=True)
    print()
    print('Cleaning Text from Training Set')
    for i in tqdm(df_training_set.index):
        df_training_set.at[i, 'cleaned_text'] = tc_class.clean(df_training_set.at[i, Train_set_Text], ['dot', 'at_user', 'hashtag', 
                                                                             'punc', 'html_link', 'pic_link', 'lower', 
                                                                             'numbers', 'stop_words', 'new_line'])
    print()
    
    
    remove_set = set()
    print('Checking for overlap')
    for i in tqdm(df_training_set.index):
        remove_indices = df_coll_set[df_coll_set['cleaned_text'] == df_training_set.at[i, 'cleaned_text']].index 
        if len(remove_indices) > 0:
            for one in remove_indices:
                remove_set.add(one)
      
    print('Removing tweets')
    df_coll_set = df_coll_set.drop(list(remove_set))
    df_coll_set.reset_index(drop=True, inplace=True)
    print(len(df_coll_set))
    print()

#does not add duplicate tweets to test set if set to False
if remove_duplicates == True:
    print('Removing duplicate tweets')
    df_coll_set = df_coll_set.drop_duplicates(subset='cleaned_text')
    df_coll_set.reset_index(drop=True, inplace=True)
    print(len(df_coll_set))
    print()

#does not add tweets outside of time frame to the test set if set to True
if make_only_in_time_frame == True:
    print('Removing tweets not during time interval ' + date_begin + " -- " + date_end)
    df_coll_set['seconds'] = df_coll_set[Date].map(conv_date_seconds)

    #df holds seconds since epoch
    #now convert 'to' and 'from' to seconds instead of '2017-09-28'

    to_obj = datetime.strptime(date_end, '%Y-%m-%d')
    from_obj = datetime.strptime(date_begin, '%Y-%m-%d')
    date_end = int(to_obj.timestamp())
    date_begin = int(from_obj.timestamp())

    #filter df to seconds between to and from then drop seconds column

    df_coll_set = df_coll_set[(df_coll_set['seconds'] <= date_end) & (df_coll_set['seconds'] >= date_begin)]
    #df_coll_set = df_coll_set[df_coll_set['seconds'] == date_end]
    df_coll_set.drop(columns=['seconds'], inplace=True)
    df_coll_set.reset_index(drop=True, inplace=True)
    print(len(df_coll_set))
    print()
    
print(len(df_coll_set))
df_coll_set = df_coll_set.drop([i for i in df_coll_set.index if df_coll_set.at[i, Date] == None])
df_coll_set = df_coll_set.drop([i for i in df_coll_set.index if df_coll_set.at[i, Text] == None])
df_coll_set = df_coll_set.drop([i for i in df_coll_set.index if df_coll_set.at[i, ID] == None])
df_coll_set.reset_index(drop=True, inplace=True)
print(len(df_coll_set))
    
list1=[]

while 1:
    if len(list1) == num or len(list1) == len(df_coll_set):
        break
    r = random.randint(0,len(df_coll_set))
    if r not in list1: 
        list1.append(r)  
        
#make sure this prints out the number of desired tweets
df_new_set['Date'] = None
df_new_set['ID'] = None
df_new_set['Text'] = None
    
k = 0
print('Creating csv')
for i in tqdm(df_coll_set.index):
    if i in list1:
        
        date = parser.parse(df_coll_set.at[i, Date])
        str_date = date.strftime("%Y-%m-%d")
        df_new_set.at[k, 'Date'] = str_date
        df_new_set.at[k, 'ID'] = '="' + str(df_coll_set.at[i, ID]) + '"'
        df_new_set.at[k, 'Text'] = df_coll_set.at[i, Text]
        k += 1
        
df_new_set.set_index('Date', inplace=True)
df_new_set.to_excel(xlsx_name)

print('Number of tweets in the csv is ' + str(len(df_new_set)))
print('Done')

5000

Cleaning Text from Collection Set
Removing retweets
Removing any tweets not in english


100%|██████████| 5000/5000 [04:21<00:00, 20.83it/s]


size of the Data Frame after is: 4158

Removing irrelevant tweets
           text Is_Relevant_Or_Not
0  Not Relevant         irrelevant
1      Relevant           relevant
predicting relevance
798

Removing retweets



798

Removing duplicate tweets
728

728
728
Creating csv



Number of tweets in the csv is 727
Done
